# matching models solved by linear programming

In [76]:
# SETUP
using JuMP
using GLPK
using Random
using LinearAlgebra

we start with many-to-many matching with no noise

In [79]:
no_w = 12
no_f = 4

q = [2,2,2,2]

S = repeat(reshape(1:no_w,:,1),1,no_f) # Supply matrix

# S[:,1] .+= 1
display(S)

# # model specification 
model = Model(GLPK.Optimizer) 

@variable(model, 0 <= μ[1:no_w, 1:no_f] <= 1)

@objective(model, Max, sum(S[i,f] * μ[i,f] for i in 1:no_w, f in 1:no_f))

worker = @constraint(model, [i=1:no_w], sum(μ[i,f] for f in 1:no_f) <= 1)
firm = @constraint(model, [f=1:no_f], sum(μ[i,f] for i in 1:no_w) <= q[f])

optimize!(model)

μ_opt = value.(μ)
println("Optimal matching is:", round.(μ_opt))

worker_value = zeros(no_w)
firm_value = zeros(no_f)

for i in 1:no_w
    worker_value[i] = shadow_price(worker[i])
end

for f in 1:no_f
    firm_value[f] = shadow_price(firm[f])
end

print(worker_value)
print(firm_value)

# noted that workers extract all their value from the firms
# and the one firm that has additional value also extract it 

12×4 Matrix{Int64}:
  1   1   1   1
  2   2   2   2
  3   3   3   3
  4   4   4   4
  5   5   5   5
  6   6   6   6
  7   7   7   7
  8   8   8   8
  9   9   9   9
 10  10  10  10
 11  11  11  11
 12  12  12  12

Optimal matching is:[0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 1.0; 0.0 0.0 1.0 0.0; 0.0 0.0 1.0 0.0; 0.0 1.0 0.0 0.0; 0.0 1.0 0.0 0.0; 1.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0][4.0, 4.0, 4.0, 4.0]

next we explore a many-to-many matching where there are more workers than vacancies 

we should see that:
(1) unemployed workers are of lower values compared to the mean 
(2) as no.worker/no.vacancy decrease the average quality of employed worker increases 
